In [1]:
!pip install antlr4-python3-runtime

In [2]:
#!pip install anytree

In [3]:
from antlr4 import *
from antlr4.tree.Trees import Trees
from anytree import Node, RenderTree

Создаем Lexer, Parser с помощью ANTLR:

In [4]:
!cd /Users/elena/ANTLR-4.13.0 && \
java -Xmx500M -cp "/Users/elena/ANTLR-4.13.0/antlr-4.13.0-complete.jar" org.antlr.v4.Tool -Dlanguage=Python3 SelectSQL.g4

In [5]:
from SelectSQLLexer import SelectSQLLexer
from SelectSQLParser import SelectSQLParser

Функция для парсера:

In [6]:
def sql_parser(select_statement):
    
    infile=InputStream(select_statement)
    infile.consume()
    lexer = SelectSQLLexer(infile)
    stream = CommonTokenStream(lexer)
    parser = SelectSQLParser(stream)
    tree = parser.select_statement()
    
    print(Trees.toStringTree(tree, None, parser))
    
    return tree

Функция для красивого вывода дерева:

In [7]:
def create_parse_tree(node):
    if isinstance(node, TerminalNode):
        return Node(node.getText())
    else:
        parse_tree = Node(node.getText())
        for child in node.children:
            child_node = create_parse_tree(child)
            child_node.parent = parse_tree
        return parse_tree

Eg.1

In [8]:
sel_st = ''' SELECT *
            FROM call
            ORDER BY call.employee_id, call.start_time ASC;'''

In [9]:
tree = sql_parser(sel_st)

(select_statement (select_clause SELECT (select_list *)) (from_clause FROM (table_list (table_name call))) (order_by_clause ORDER BY (column_name_list (table_column (table_name call) . (column_name employee_id)) , (table_column (table_name call) . (column_name start_time))) ASC) ;)


In [10]:
root = create_parse_tree(tree)
for pre, fill, node in RenderTree(root):
    print(f"{pre}{node.name}")

SELECT*FROMcallORDERBYcall.employee_id,call.start_timeASC;
├── SELECT*
│   ├── SELECT
│   └── *
│       └── *
├── FROMcall
│   ├── FROM
│   └── call
│       └── call
│           └── call
├── ORDERBYcall.employee_id,call.start_timeASC
│   ├── ORDER
│   ├── BY
│   ├── call.employee_id,call.start_time
│   │   ├── call.employee_id
│   │   │   ├── call
│   │   │   │   └── call
│   │   │   ├── .
│   │   │   └── employee_id
│   │   │       └── employee_id
│   │   ├── ,
│   │   └── call.start_time
│   │       ├── call
│   │       │   └── call
│   │       ├── .
│   │       └── start_time
│   │           └── start_time
│   └── ASC
└── ;


Eg.2

In [11]:
sel_st1 = ''' WITH highest AS (SELECT table.branch, date, MAX(unit_price) AS highest_price 
             FROM sales GROUP BY branch, date)
             SELECT sales.age, h.highest_price FROM sales 
             JOIN highest AS h ON sales.branch = h.branch AND sales.date = h.date 
             ORDER BY table.col, age DESC
'''

In [12]:
tree = sql_parser(sel_st1)

(select_statement (select_with_cte WITH (column_name_list (column_name highest)) AS ( (select_statement (select_clause SELECT (select_list (select_item (expression (table_column (table_name table) . (column_name branch)))) , (select_item (expression (column_name date))) , (select_item (expression (aggregate_function MAX ( (column_name unit_price) ))) AS (column_name highest_price)))) (from_clause FROM (table_list (table_name sales))) (group_by_clause GROUP BY (column_name_list (column_name branch) , (column_name date)))) )) (select_clause SELECT (select_list (select_item (expression (table_column (table_name sales) . (column_name age)))) , (select_item (expression (table_column (table_name h) . (column_name highest_price)))))) (from_clause FROM (table_list (table_name sales))) (join_list (join_clause JOIN (table_name highest) AS (table_name h) ON (join_condition (table_column (table_name sales) . (column_name branch)) (comparison_operator =) (table_column (table_name h) . (column_name 

In [13]:
root = create_parse_tree(tree)
for pre, fill, node in RenderTree(root):
    print(f"{pre}{node.name}")

WITHhighestAS(SELECTtable.branch,date,MAX(unit_price)AShighest_priceFROMsalesGROUPBYbranch,date)SELECTsales.age,h.highest_priceFROMsalesJOINhighestAShONsales.branch=h.branchANDsales.date=h.dateORDERBYtable.col,ageDESC
├── WITHhighestAS(SELECTtable.branch,date,MAX(unit_price)AShighest_priceFROMsalesGROUPBYbranch,date)
│   ├── WITH
│   ├── highest
│   │   └── highest
│   │       └── highest
│   ├── AS
│   ├── (
│   ├── SELECTtable.branch,date,MAX(unit_price)AShighest_priceFROMsalesGROUPBYbranch,date
│   │   ├── SELECTtable.branch,date,MAX(unit_price)AShighest_price
│   │   │   ├── SELECT
│   │   │   └── table.branch,date,MAX(unit_price)AShighest_price
│   │   │       ├── table.branch
│   │   │       │   └── table.branch
│   │   │       │       └── table.branch
│   │   │       │           ├── table
│   │   │       │           │   └── table
│   │   │       │           ├── .
│   │   │       │           └── branch
│   │   │       │               └── branch
│   │   │       ├── ,
│   │   │    

Eg. 3

In [14]:
sel_st2 = ''' WITH highest AS (SELECT table.branch, date, MAX(unit_price) AS highest_price 
             FROM sales GROUP BY branch, date)
             SELECT sales.age, h.highest_price 
             FROM sales LEFT JOIN highest AS h ON sales.branch = h.branch AND sales.date = h.date
             WHERE sales.num >= 100 OR sales.manager = "John" AND sales.revenue IS NOT NULL
             ORDER BY table.col, age DESC
             LIMIT 10;
'''

In [15]:
tree = sql_parser(sel_st2)

(select_statement (select_with_cte WITH (column_name_list (column_name highest)) AS ( (select_statement (select_clause SELECT (select_list (select_item (expression (table_column (table_name table) . (column_name branch)))) , (select_item (expression (column_name date))) , (select_item (expression (aggregate_function MAX ( (column_name unit_price) ))) AS (column_name highest_price)))) (from_clause FROM (table_list (table_name sales))) (group_by_clause GROUP BY (column_name_list (column_name branch) , (column_name date)))) )) (select_clause SELECT (select_list (select_item (expression (table_column (table_name sales) . (column_name age)))) , (select_item (expression (table_column (table_name h) . (column_name highest_price)))))) (from_clause FROM (table_list (table_name sales))) (join_list (join_clause LEFT JOIN (table_name highest) AS (table_name h) ON (join_condition (table_column (table_name sales) . (column_name branch)) (comparison_operator =) (table_column (table_name h) . (column_

In [16]:
root = create_parse_tree(tree)
for pre, fill, node in RenderTree(root):
    print(f"{pre}{node.name}")

WITHhighestAS(SELECTtable.branch,date,MAX(unit_price)AShighest_priceFROMsalesGROUPBYbranch,date)SELECTsales.age,h.highest_priceFROMsalesLEFTJOINhighestAShONsales.branch=h.branchANDsales.date=h.dateWHEREsales.num>=100ORsales.manager="John"ANDsales.revenueISNOTNULLORDERBYtable.col,ageDESCLIMIT10;
├── WITHhighestAS(SELECTtable.branch,date,MAX(unit_price)AShighest_priceFROMsalesGROUPBYbranch,date)
│   ├── WITH
│   ├── highest
│   │   └── highest
│   │       └── highest
│   ├── AS
│   ├── (
│   ├── SELECTtable.branch,date,MAX(unit_price)AShighest_priceFROMsalesGROUPBYbranch,date
│   │   ├── SELECTtable.branch,date,MAX(unit_price)AShighest_price
│   │   │   ├── SELECT
│   │   │   └── table.branch,date,MAX(unit_price)AShighest_price
│   │   │       ├── table.branch
│   │   │       │   └── table.branch
│   │   │       │       └── table.branch
│   │   │       │           ├── table
│   │   │       │           │   └── table
│   │   │       │           ├── .
│   │   │       │           └── branch
